In [ ]:
import json
from functools import partial

import pandas as pd
import pyproj
import shapely.ops as ops
from shapely.geometry import shape

from city_conf import city_mappings

## Map data results

In [ ]:
def find_polygon_area(geojson):
    geom = shape(geojson)
    geom_area = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj(init='EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=geom.bounds[1],
                lat_2=geom.bounds[3]
            )
        ),
        geom)
    return geom_area.area / 1e6

In [ ]:
city_records = []
for country_map in city_mappings:
    for city in city_mappings[country_map]:
        city_name = list(city.keys())[0]
        osm_id = city[city_name]["osm_id"]
        try:
            with open(f"results/{city_name}.json", "r") as f:
                city_record = json.load(f)

            with open(f"city_polygons/{city_name.lower()}_polygon.geojson") as f:
                city_polygon = json.load(f)

            city_record["osm_id"] = osm_id
            city_record["area_km2"] = find_polygon_area(city_polygon)
            city_records.append(city_record)
        except:
            continue

In [ ]:
df = pd.DataFrame(city_records)

In [ ]:
city_records_with_decay = []
for country_map in city_mappings:
    for city in city_mappings[country_map]:
        city_name = list(city.keys())[0]
        osm_id = city[city_name]["osm_id"]
        try:
            with open(f"results/{city_name}_decay.json", "r") as f:
                city_record = json.load(f)

            with open(f"city_polygons/{city_name.lower()}_polygon.geojson") as f:
                city_polygon = json.load(f)

            city_record["osm_id"] = osm_id
            city_record["area_km2"] = find_polygon_area(city_polygon)
            city_records_with_decay.append(city_record)
        except:
            continue

In [ ]:
df_decay = pd.DataFrame(city_records_with_decay)

In [ ]:
df["overall_road_length"] = df["total_cycling_road_length"] + df["total_road_length"]
df_decay["overall_road_length"] = df_decay["total_cycling_road_length"] + df_decay["total_road_length"]

In [ ]:
df["cycle_road_share"] = df["total_cycling_road_length"] / df["overall_road_length"]
df_decay["cycle_road_share"] = df_decay["total_cycling_road_length"] / df_decay["overall_road_length"]

In [ ]:
df["cycle_track_share"] = df["total_cycle_track_length"] / df["overall_road_length"]
df_decay["cycle_track_share"] = df_decay["total_cycle_track_length"] / df_decay["overall_road_length"]

In [ ]:
df["segregated_cycle_track_share"] = df["total_segregated_cycle_track_length"] / df["overall_road_length"]
df_decay["segregated_cycle_track_share"] = df_decay["total_segregated_cycle_track_length"] / df_decay[
    "overall_road_length"]

In [ ]:
df["rank_cycle_road_share"] = df["cycle_road_share"].rank(ascending=False).astype(int)
df["rank_cycle_track_share"] = df["cycle_track_share"].rank(ascending=False).astype(int)
df["rank_segregated_cycle_track_share"] = df["segregated_cycle_track_share"].rank(ascending=False).astype(int)

In [ ]:
df_decay["rank_cycle_road_share"] = df_decay["cycle_road_share"].rank(ascending=False).astype(int)
df_decay["rank_cycle_track_share"] = df_decay["cycle_track_share"].rank(ascending=False).astype(int)
df_decay["rank_segregated_cycle_track_share"] = df_decay["segregated_cycle_track_share"].rank(ascending=False).astype(
    int)

In [ ]:
merged = df.merge(df_decay, on=["city_name", "osm_id", "area_km2"], suffixes=["", "_decayed"])

# merged["overall_score"] = merged["cycle_road_share_decayed"] * merged["cycle_track_share_decayed"] #* merged[
#     "segregated_cycle_track_share_decayed"]
merged["overall_score"] = merged["cycle_road_share"] * merged["segregated_cycle_track_share"]
merged["overall_rank"] = merged["overall_score"].rank(ascending=False).astype(int)

In [ ]:
merged["parking_per_km2"] = merged['parking_counter'] / merged["area_km2"]

In [ ]:
final = merged[
    ["city_name", "osm_id", "area_km2",
     "total_road_length",
     "total_cycling_road_length",
     "cycle_road_share",
#      "cycle_track_share",
     "segregated_cycle_track_share",
     "rank_cycle_road_share",
#      "rank_cycle_track_share",
     "rank_segregated_cycle_track_share",
     "rank_cycle_road_share_decayed",
#      "rank_cycle_track_share_decayed",
     "rank_segregated_cycle_track_share_decayed",
     "parking_per_km2",
     "overall_rank"]].round(3)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
final.sort_values("overall_rank", ascending=True).head(100)

In [ ]:
final[final.city_name == "Tallinn"]

## Road network

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
from pyrosm import OSM

In [ ]:
osm_city = "Tallinn"

In [ ]:
filepath = f'extracted_maps/{osm_city}.pbf'
osm = OSM(filepath)
# nodes_driving, edges_driving = osm.get_network(nodes=True, network_type="cycling")
# nodes_all, edges_all = osm.get_network(nodes=True, network_type="all")
drive_net = osm.get_network(network_type="cycling")

In [ ]:
drive_net.sort_values("length")

In [ ]:
subset = drive_net[drive_net["id"].isin(list(way_ids.keys()))]

In [ ]:
subset["length"].sum() / drive_net["length"].sum()

In [ ]:
lanes = subset[subset["highway"] != "cycleway"]

In [ ]:
subset.shape

In [ ]:
subset["length"].sum()

In [ ]:
lanes["length"].sum()

In [ ]:
subset.plot(figsize=(35,35), column="highway", legend=True)

In [ ]:
subset.sort_values("length")

In [ ]:
paths = subset[subset["highway"].isin(["path", "footway"])]

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

In [ ]:
expanded = paths.explode()

In [ ]:
# colors = ["red" if x == "footway" else "blue" for x in expanded["highway"]]

# lngs = []
# lats = []
# for geom in expanded["geometry"]:
#     (start_lng, start_lat), (end_lng, end_lat) = geom.coords
#     lngs.append((start_lng, end_lng))
#     lats.append((start_lat, end_lat))

In [ ]:
# from tqdm import tqdm

In [ ]:
# for i in tqdm(range(len(lngs))):
#     plt.plot(lngs[i], lats[i], linewidth=3.0, color=colors[i])    
# mplleaflet.show()

In [ ]:
# TODO remove footways
# TODO review oneway weighting
# TODO make cool plots

In [ ]:
# expanded.groupby("highway")["length"].sum() / expanded["length"].sum()

In [ ]:
import pickle
with open(f"/Users/martin/Desktop/fun/cyclorank/results/{osm_city}_way_ids.pkl", "rb") as f:
    way_ids = pickle.load(f)

In [ ]:
way_to_coef = {}
i = 0

for w in way_ids:
    coef = way_ids[w]["weighted_distance"] / way_ids[w]["raw_distance"]
    way_to_coef[w] = coef

In [ ]:
two_lanes = drive_net[drive_net.id.isin(way_ids)]

In [ ]:
expanded = two_lanes.explode()

In [ ]:
from colour import Color
red = Color("gray")
colors = list(red.range_to(Color("red"),11))

In [ ]:
def color_map(coef): return colors[int(round(coef*10))]

In [ ]:
way_to_coord = {}

In [ ]:
lngs = []
lats = []
ids = []
for geom, way_id in zip(expanded["geometry"], expanded["id"]):
    (start_lng, start_lat), (end_lng, end_lat) = geom.coords
    if way_id not in way_to_coord:
        way_to_coord[way_id] = (start_lat, start_lng)
    lngs.append((start_lng, end_lng))
    lats.append((start_lat, end_lat))
    ids.append(way_id)

In [ ]:
from shapely.geometry import Point, shape, Polygon, MultiPolygon
from functools import partial
import random
import pyproj
from shapely.ops import transform
import matplotlib.pyplot as plt

city_centroid = [24.7453688, 59.4372155]

In [ ]:
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

def create_circle_around_coord(lat, lon, km):
    b = geodesic_point_buffer(lat, lon, km)
    circle_lats = [x[1] for x in b]
    circle_lngs = [x[0] for x in b]
    return circle_lats, circle_lngs, b

In [ ]:
circle = create_circle_around_coord(city_centroid[1], city_centroid[0], 8)

In [ ]:
plt.scatter(city_centroid[0], city_centroid[1], color='red', marker='x', s=500)
plt.plot(circle[1], circle[0], '-.', color="#812a2a")
for i in tqdm(range(len(lngs))):
    color = str(color_map(way_to_coef[ids[i]]))
    plt.plot(lngs[i], lats[i], linewidth=2.0, color=color)
mplleaflet.show(tiles="cartodb_positron")

## Routing results

In [ ]:
raise ValueError

### Sample coordinates around centroid

In [ ]:
city_name = "Bucharest"

In [ ]:
def random_points_within(circle_polygon, city_polygon, num_points):
    min_x, min_y, max_x, max_y = circle_polygon.bounds

    points = []

    while len(points) < num_points:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if random_point.within(circle_polygon):
            if random_point.within(city_polygon):
                points.append(random_point)

    return points





def create_circle_around_coord(lat, lon, km):
    b = geodesic_point_buffer(lat, lon, km)
    circle_lats = [x[1] for x in b]
    circle_lngs = [x[0] for x in b]
    return circle_lats, circle_lngs, b


def load_city_shapes(city_name):
    with open(f"city_polygons/{city_name.lower()}_polygon.geojson") as f:
        city_json = json.load(f)

    city_polygon = city_json["geometries"][0]
    city_shape = shape(city_polygon)
            
    with open(f"city_polygons/{city_name.lower()}.geojson") as f:
        city_json = json.load(f)

    city_centroid_shape = shape(city_json)
    city_centroid = Point((city_centroid_shape.centroid.y, city_centroid_shape.centroid.x))
    
    xx, yy = city_centroid.x, city_centroid.y

    circle_lats, circle_lngs, b = create_circle_around_coord(yy, xx, 7.5)

    circle_polygon = Polygon(b)
    
    shapes = {
        "circle_polygon": circle_polygon,
        "city_polygon": city_shape,
        "city_centroid": city_centroid,
        "circle_lats": circle_lats,
        "circle_lngs": circle_lngs,
        "centroid_x": xx,
        "centroid_y": yy
    }
    
    return shapes

In [ ]:
shapes = load_city_shapes(city_name)

In [ ]:
points = random_points_within(circle_polygon=shapes["circle_polygon"], city_polygon=shapes["city_polygon"], num_points=50)

In [ ]:
point_lats = [pt.x for pt in points]
point_lngs = [pt.y for pt in points]

In [ ]:
circle_lngs, circle_lats = shapes["circle_polygon"].exterior.xy

In [ ]:
plt.plot(circle_lngs, circle_lats, '--')
plt.scatter(shapes["centroid_x"], shapes["centroid_y"], color='red')
plt.scatter(point_lats, point_lngs, alpha=0.3, color="gray")
try:
    x, y = shapes["city_polygon"].buffer(0).exterior.xy
    plt.plot(x, y, color="blue")
except AttributeError: # MultiPolygon
    g = [x.buffer(0) for x in shapes["city_polygon"].buffer(0).geoms]
    for i in range(len(g)):
        x, y = g[i].exterior.xy
        plt.plot(x, y, color="blue")
plt.show()

## Routing results

In [ ]:
raise ValueError

In [ ]:
city_name = "Bucharest"
import pandas as pd

In [ ]:
results = {}
for vehicle_type in ["car", "bike"]:
    with open(f"routing/routing_results/{city_name}_{vehicle_type}.json", "r") as f:
        results[vehicle_type] = pd.DataFrame(json.load(f))

In [ ]:
results["car"]

In [ ]:
results["bike"]["distance"]

In [ ]:
merged = results["car"].merge(results["bike"], suffixes=["_car", "_bike"], left_index=True, right_index=True)

In [ ]:
merged

In [ ]:
merged["duration_delta"] = merged["duration_bike"] / merged["duration_car"]
merged["distance_delta"] = merged["distance_bike"] / merged["distance_car"]

In [ ]:
merged

In [ ]:
plt.hist(merged["distance_car"], bins=20, alpha=0.3)
plt.hist(merged["distance_bike"], bins=20, alpha=0.3)
plt.show()

In [ ]:
plt.hist(merged["duration_car"], bins=20, alpha=0.3)
plt.hist(merged["duration_bike"], bins=20, alpha=0.3)
plt.show()

In [ ]:
merged["distance_car"].median()

In [ ]:
merged["distance_bike"].median()

In [ ]:
merged["duration_bike"].median()

In [ ]:
merged["distance_delta"].median()

In [ ]:
merged["duration_delta"].median()